In [1]:
import pandas as pd
import numpy as np
from os.path import join
import re
from multiprocessing import Pool
import pickle

# 1. Load variables and dataframe

In [2]:
input_file = join('/Users', 'Toavina', 'githubdata',
                  '7.regetting_gh_events_data', '3.ghevents_df',
                  'hnusers_ghevents_df.pkl')

save_folder = join('/Users', 'Toavina', 'githubdata',
                   '8.transposing_gh_events', '1.saved_df')

save_filename = 'gh_events_bymonthusertype_df.pkl'

In [3]:
ghevents_df = pickle.load(open(input_file, 'rb'))

In [5]:
print(
    'Setting relevant columns as categorical, boolean and date columns to save memory and ease manipulation'
)
ghevents_df['type'] = ghevents_df['type'].astype('category')
ghevents_df['own_repo'] = ghevents_df['own_repo'].astype('bool')
ghevents_df['org_event'] = ghevents_df['org_event'].astype('bool')


def amend_date_col(date_col):
    """Removes the UTC string for a date column so it can be converted to numpy datetime"""

    date_col = re.sub(' UTC', '', date_col)

    return date_col


ghevents_df['created_at'] = ghevents_df['created_at'].apply(
    amend_date_col).astype('datetime64')

Setting relevant columns as categorical, boolean and date columns to save memory and ease manipulation


In [46]:
print('Generating pivot table of event counts by user, type and month')
ghevents_pivoted = ghevents_df.groupby([
    'actor_login', 'type', pd.Grouper(
        freq='1M', key='created_at')
]).count().unstack('actor_login').transpose().loc['created_at']

# Alternative command
# ghevents_df.groupby(['actor_login','type','year_month_created']).count().unstack('actor_login').transpose().loc['created_at']

In [47]:
print('Filling NAs as zero and transforming counds to integers')
ghevents_pivoted.fillna(0, inplace=True)
ghevents_pivoted = ghevents_pivoted.astype('int')

In [59]:
print('Saving dataframe to ' + join(save_folder,save_filename))
pickle.dump(ghevents_pivoted, open(join(save_folder,save_filename),'wb'))

Saving dataframe to /Users/Toavina/githubdata/8.transposing_gh_events/1.saved_df/gh_events_bymonthusertype_df.pkl
